In [469]:
import pandas as pd

In [498]:
# Lecture du CSV des demandes
cols = ["#", "Tâche parente", "Sujet", "Auteur", "Début", "% réalisé"]
issues_df = pd.read_csv("issues.csv", encoding="latin", sep=";", usecols=cols)
issues_df

,#,Tâche parente,Sujet,Auteur,Début,% réalisé
0,26,NaN,Gestion de projet / Management,Erwan SZYMANSKI,16/02/2020,48
1,76,26.0,Gestion Redmine,Erwan SZYMANSKI,19/02/2020,100
2,123,26.0,Céer snippet javascript pour exporter heures R...,Erwan SZYMANSKI,01/03/2020,20
3,126,26.0,Reunions,Erwan SZYMANSKI,01/03/2020,100
4,1257,126.0,Préparer réunion 6 Erwan,Erwan SZYMANSKI,10/04/2020,100
...,...,...,...,...,...,...
239,1288,374.0,Consulting 14 : Architecture diag séquence et ...,Erwan SZYMANSKI,NaN,0
240,1350,374.0,Consulting 15: Vérifications des diagrammes de...,Erwan SZYMANSKI,NaN,0
241,678,NaN,Soudure Robot,Pierre PROUTEAU,NaN,100
242,1140,678.0,Cablages Robot,Pierre PROUTEAU,NaN,100


In [471]:
first_names = list(issues_df["Auteur"].apply(lambda x: x.split(" ")[0].lower()).unique())
first_names

['erwan',
 'clément',
 'tristan',
 'quentin',
 'alexy',
 'mathieu',
 'pierre',
 'julien']

In [472]:
# Lecture des CSV du temps passé
cols = ["Utilisateur", "Demande", "Heures"]
timelog_df = pd.read_csv("timelog.csv", encoding="latin", sep=";", usecols=cols)
timelog_df["Prénom"] = timelog_df["Utilisateur"].apply(lambda x: x.split(" ")[0].lower())
timelog_df

,Utilisateur,Demande,Heures,Prénom
0,Quentin TOMAT,Réunion #1350: Consulting 15: Vérifications de...,"1,50",quentin
1,Mathieu FREIRE,Réunion #1350: Consulting 15: Vérifications de...,"1,50",mathieu
2,Clément TOUPIN,Réunion #1350: Consulting 15: Vérifications de...,"1,50",clément
3,Alexy DEBUS,Réunion #1350: Consulting 15: Vérifications de...,"1,50",alexy
4,Erwan SZYMANSKI,Tâche #1255: Superviser les explorations techn...,"0,75",erwan
...,...,...,...,...
746,Erwan SZYMANSKI,Tâche #49: Autoformer LateX Erwan,"0,50",erwan
747,Erwan SZYMANSKI,Tâche #47: Créer fichier tutoriel au remplissa...,"0,42",erwan
748,Erwan SZYMANSKI,Tâche #49: Autoformer LateX Erwan,"0,33",erwan
749,Erwan SZYMANSKI,Tâche #28: Créer tâches principales Redmine (p...,"0,25",erwan


In [493]:
# Ajout d'une colonne avec le numéro précis de la tâche
import re

def get_task_number(demande):
    result = re.search('#(\d{0,9}):', demande)
    return result.group(1)

timelog_df["Tâche"] = timelog_df["Demande"].apply(get_task_number)
timelog_df.loc[timelog_df["Heures"].str.isnumeric() == False]

,Utilisateur,Demande,Heures,Prénom,Tâche
0,Quentin TOMAT,Réunion #1350: Consulting 15: Vérifications de...,"1,50",quentin,1350
1,Mathieu FREIRE,Réunion #1350: Consulting 15: Vérifications de...,"1,50",mathieu,1350
2,Clément TOUPIN,Réunion #1350: Consulting 15: Vérifications de...,"1,50",clément,1350
3,Alexy DEBUS,Réunion #1350: Consulting 15: Vérifications de...,"1,50",alexy,1350
4,Erwan SZYMANSKI,Tâche #1255: Superviser les explorations techn...,"0,75",erwan,1255
...,...,...,...,...,...
746,Erwan SZYMANSKI,Tâche #49: Autoformer LateX Erwan,"0,50",erwan,49
747,Erwan SZYMANSKI,Tâche #47: Créer fichier tutoriel au remplissa...,"0,42",erwan,47
748,Erwan SZYMANSKI,Tâche #49: Autoformer LateX Erwan,"0,33",erwan,49
749,Erwan SZYMANSKI,Tâche #28: Créer tâches principales Redmine (p...,"0,25",erwan,28


In [474]:
# Connexion au repo git

from git import Repo
import os

repo = Repo(".", search_parent_directories=True)

commits = list(repo.iter_commits('master'))

In [475]:
# Création d'un dataframe de commmits

commits_df = pd.DataFrame()

def get_commit_task_number(commit):
    result = re.findall('#(\d*)', commit)
    if result is None:
        return None
    return result

commits_df["Reference"] = list(map(lambda commit: commit.hexsha[:8], commits))
commits_df["Auteur"] = list(map(lambda commit: commit.author.name, commits))
commits_df["Message"] = list(map(lambda commit: commit.message, commits))
commits_df["Tâches"] = commits_df["Message"].apply(get_commit_task_number)
commits_df["Prénom"] = commits_df["Auteur"].apply(lambda x: x.split(" ")[0].lower())

commits_df

,Reference,Auteur,Message,Tâches,Prénom
0,01c2a39b,Benjamin GONDANGE,[#1342] Création d'un script Python pour autom...,[1342],benjamin
1,7552b6a4,Thomas,[#1187] Modif. du diagramme de classes\n,[1187],thomas
2,3a01f67e,Thomas,[#1032] Ajout des révisions\n,[1032],thomas
3,412229f4,Gatien,[#1343] Plublication de la version PAQL audit\n,[1343],gatien
4,06e79058,Thomas,[#1187] Modifications Machine à Etats\n,[1187],thomas
...,...,...,...,...,...
243,484f26d6,Benjamin,Redact specs skeleton\n\nThis skeleton will be...,[],benjamin
244,3c55fba4,Benjamin,Add gitignore\n\nThis file automatically ignor...,[],benjamin
245,31a9b919,Gatien,Depot de la version 0.3 du PAQL\n\nEtablisseme...,[],gatien
246,bc924d73,Gaston,Redact CBOE\n\nRedaction of CBOE to prepare th...,[],gaston


In [476]:
def time_for_task_only(task, first_name):
    if type(task) != str:
        task = str(task)
        
    time = 0
    
    timelogs_for_task = timelog_df.loc[(timelog_df["Tâche"] == task) & (timelog_df["Prénom"] == first_name)]

    for index, row in timelogs_for_task.iterrows():
        heures = row["Heures"].replace(",", ".")
        time += float(heures)
    
    return time

In [500]:
time_for_task_only(49, "erwan")

3.08

In [477]:
# Recusrive function that computes the amount of time per task with subtasks

def time_for_children_tasks(task, first_name):
    key_task = "Temps passé tâche {}".format(first_name)
    if type(task) != str:
        task = str(task)
    
    children_tasks = issues_df.loc[issues_df["Tâche parente"] == int(task)]

    if children_tasks.empty:
        return 0
    
    time = 0
        
    task_row = issues_df.loc[issues_df["#"] == int(task)].iloc[0]
    for index, row in children_tasks.iterrows():
        time += row[key_task]
        time += time_for_children_tasks(row["#"], first_name)
    
    return time

In [478]:
for first_name in first_names:
    key_task = "Temps passé tâche {}".format(first_name)
    issues_df[key_task] = issues_df.apply(lambda x: time_for_task_only(x["#"], first_name), axis=1)

In [479]:
for first_name in first_names:
    key_children_tasks = "Temps passé sous-tâches {}".format(first_name)
    issues_df[key_children_tasks] = issues_df.apply(lambda x: time_for_children_tasks(x["#"], first_name), axis=1)

In [480]:
for first_name in first_names:
    key_children = "Temps passé sous-tâches {}".format(first_name)
    key_task = "Temps passé tâche {}".format(first_name)
    kay_all = "Temps passé total {}".format(first_name)
    issues_df[kay_all] = issues_df[key_task] + issues_df[key_children]

In [481]:
def commits_for_task(task, first_name):
    if type(task) != str:
        task = str(task)
    
    commits = []
    
    mask_task = commits_df["Tâches"].apply(lambda x: task in x)
    mask_first_name = commits_df["Prénom"].apply(lambda x: x == first_name)
    
    mask = mask_task & mask_first_name
    
    commit_list_for_task = commits_df.loc[mask]
    
    for index, row in commit_list_for_task.iterrows():
        commits.append(row["Reference"])
        
    children_tasks = issues_df.loc[issues_df["Tâche parente"] == int(task)]

    for index, row in children_tasks.iterrows():
        commits += commits_for_task(row["#"], first_name)
    
    return commits

In [482]:
for first_name in first_names:
    key = "Commits {}".format(first_name)
    issues_df[key] = issues_df.apply(lambda x: commits_for_task(x["#"], first_name), axis=1)

In [483]:
issues_df

,#,Tâche parente,Sujet,Auteur,Début,% réalisé,Temps passé tâche erwan,Temps passé tâche clément,Temps passé tâche tristan,Temps passé tâche quentin,...,Temps passé total pierre,Temps passé total julien,Commits erwan,Commits clément,Commits tristan,Commits quentin,Commits alexy,Commits mathieu,Commits pierre,Commits julien
0,26,NaN,Gestion de projet / Management,Erwan SZYMANSKI,16/02/2020,48,0.00,0.0,0.0,0.0,...,0.0,0.0,[],[],[],[],[],[],[],[]
1,76,26.0,Gestion Redmine,Erwan SZYMANSKI,19/02/2020,100,0.00,0.0,0.0,0.0,...,0.0,0.0,[],[],[],[],[],[],[],[]
2,123,26.0,Céer snippet javascript pour exporter heures R...,Erwan SZYMANSKI,01/03/2020,20,3.50,0.0,0.0,0.0,...,0.0,0.0,[],[],[],[],[],[],[],[]
3,126,26.0,Reunions,Erwan SZYMANSKI,01/03/2020,100,0.17,0.0,0.0,0.0,...,0.0,0.0,[],[],[],[],[],[],[],[]
4,1257,126.0,Préparer réunion 6 Erwan,Erwan SZYMANSKI,10/04/2020,100,0.50,0.0,0.0,0.0,...,0.0,0.0,[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,1288,374.0,Consulting 14 : Architecture diag séquence et ...,Erwan SZYMANSKI,NaN,0,1.00,1.0,0.0,0.0,...,0.0,0.0,[],[],[],[],[],[],[],[]
240,1350,374.0,Consulting 15: Vérifications des diagrammes de...,Erwan SZYMANSKI,NaN,0,1.50,1.5,0.0,1.5,...,0.0,0.0,[],[],[],[],[],[],[],[]
241,678,NaN,Soudure Robot,Pierre PROUTEAU,NaN,100,0.00,0.0,0.0,0.0,...,7.5,0.0,[],[],[],[],[],[],[],[]
242,1140,678.0,Cablages Robot,Pierre PROUTEAU,NaN,100,0.00,0.0,0.0,0.0,...,3.0,0.0,[],[],[],[],[],[],[],[]


In [484]:
# Calcul des temps passés globaux

times = {}

for first_name in first_names:
    time_key = "Temps passé total {}".format(first_name)
    times[first_name] = issues_df.loc[issues_df["Tâche parente"].isnull()][time_key].sum()
    
times

{'erwan': 132.59,
 'clément': 85.91,
 'tristan': 53.33,
 'quentin': 49.66,
 'alexy': 120.34,
 'mathieu': 50.53000000000001,
 'pierre': 78.67,
 'julien': 46.34}

In [485]:
# Rédaction de la fiche synthèse

import math

def is_low_level(task):
    children_tasks = issues_df.loc[issues_df["Tâche parente"] == int(task["#"])]
    
    return children_tasks.empty

def find_low_level_children_tasks(task, first_name):
    time_spent_key = "Temps passé total {}".format(first_name)
    children_tasks = issues_df.loc[issues_df["Tâche parente"] == int(task["#"])]
    
    low_level_tasks = []
    
    for index, row in children_tasks.iterrows():
        sub_children_tasks = issues_df.loc[issues_df["Tâche parente"] == int(row["#"])]
        if sub_children_tasks.empty and row[time_spent_key] != 0:
            low_level_tasks.append(row)
    
    return low_level_tasks
    

def write_tasks(task, first_name, level=0):
    task_text = ""
    
    time_spent_key = "Temps passé total {}".format(first_name)
    time_spent_task_key = "Temps passé tâche {}".format(first_name)
    
    commits_key = "Commits {}".format(first_name)
    time_spent = task[time_spent_key]
    time_spent_task = task[time_spent_task_key]
    
    if time_spent == 0:
        return task_text
    
    if level == 0:
        task_text += "\nh2. "
    elif level == 1:
        task_text += "\nh3. "
    elif level == 2:
        task_text += "\nh4. "
    elif level == 3:
        task_text += "\nh5. "
    elif level != math.inf:
        task_text +="\n"
    
    if level < 50:
        if (time_spent_task == 0):
            task_text += "{} - {}h (total)\n".format(task["Sujet"], time_spent)
        else:
            task_text += "{} - {}h (total) - {}h (tâche)\n".format(task["Sujet"], time_spent, time_spent_task)
    else:
        task_text += "\n| #{task_id} | {start_date} | {subject} | {time_spent}h | {percentage}% | {commits} |".format(
            task_id=task["#"],
            start_date=task["Début"],
            subject=task["Sujet"],
            time_spent=time_spent,
            percentage=task["% réalisé"],
            commits=", ".join(task[commits_key])
        )
    
    if level < 4:
        task_text += "\n"
        
    children_tasks = issues_df.loc[issues_df["Tâche parente"] == int(task["#"])]
    
    low_level_tasks = pd.DataFrame(find_low_level_children_tasks(task, first_name))
    
    other_tasks = children_tasks.drop(low_level_tasks.index)
    
    if low_level_tasks.empty == False:
        task_text += "\n|_. Tâche |_. Date de début |_. Sujet |_. Temps passé |_. Pourcentage de réalisation |_. Commits |"
        for index, row in low_level_tasks.iterrows():
            task_text += write_tasks(row, first_name, math.inf)
        task_text += "\n"
    
    for index, row in other_tasks.iterrows():
        task_text += write_tasks(row, first_name, level + 1)
        
    return task_text
    
def write_wiki(first_name):
    text = "h1. Fiche synthèse de {}\n\n".format(first_name)
    text += "*Temps total passé :* {}h\n\n".format(times[first_name])
    
    parent_tasks = issues_df.loc[issues_df["Tâche parente"].isnull()]
    
    for index, row in parent_tasks.iterrows():
        text += write_tasks(row, first_name)
        text += "\n\n"
            
    return text

In [486]:
try:
    os.makedirs("syntheses")
except FileExistsError:
    pass

for first_name in first_names:
    with open("syntheses/{}.txt".format(first_name), 'w') as f:
        f.write(write_wiki(first_name))